In [1]:
import sys
import numpy as np
sys.path.append('../')
from time import sleep
from quantum_compiler import shape_functionV4 as sf
from quantum_compiler.WavetoolsV6 import Wave
from instruments.core.jsonIO import jsonFormat
from instruments.drivers.Tektronix.Tektronix_AWG import AWG5208

In [2]:
# connect to AWG
awg = AWG5208(
    inst_name='AWG5208',
    inst_address='TCPIP0::192.168.10.42::inst0::INSTR')

# help(awg)

# set sampling rate and reference clock
awg.set_sample_rate(sample_rate=1.0E9)
awg.set_extref_source(ref_freq=10E6)

Connect Instrument:
instrument: AWG5208
statement: TEKTRONIX,AWG5208,B030597,FV:6.6.0131.0


In [3]:
# # assign waveforms
# import shape_functionV4 as sf
# from WavetoolsV6 import Wave

# freq = 20e6 #delta freq
# duration = 1
# f_mod = 0.05e6 #modulation freq
# phase = 0.5*np.pi #phase difference
# time_p = phase/f_mod/(2*np.pi)
# offset = 10e-6
# marker_length = 1e-6
# x_span = 1/f_mod/2


# a = Wave(sf.const, [sf.get_x(offset, sampling_rate), False]) #constant wave
# b = Wave(sf.sine2, [sf.get_x(x_span, sampling_rate), freq]) #sine wave
# c = Wave(sf.cosine2, [sf.get_x(x_span, sampling_rate), freq]) #consine wave
# d = Wave(sf.const, [sf.get_x(x_span, sampling_rate), True]) #flux
# e = Wave(sf.const, [sf.get_x(time_p, sampling_rate), False]) #phase

# I = ~a + ~e + (~c + ~c)*duration + ~a
# Q = ~a + ~e + (~(b) + ~(-1*b))*duration + ~a
# Flux = ~a + (~(d) + ~(-1*d))*duration + ~a
# marker = ~(Wave(sf.square, [Flux.x, 8e-6, marker_length]))
# zeros = ~(Wave(sf.const, [I.x, False]))

# print(len(I))
# print(len(Flux))
# Flux.plot()
# gate = ~I / ~Q / ~Flux / ~marker / ~zeros 
# # gate.set_wire_names(['I', 'Q', 'Flux' , 'Marker', 'zeros'])
# gate.plot(size=[20,3])
# print(len(gate.y[0]), len(gate.y[1]), len(gate.y[2]), len(gate.y[3]), len(gate.y[4]))
# awg.clr_wfm()
# IQList = ['I', 'Q']
# awg.set_wfm(wfm_name=f'{IQList[0]}',
#             wfm=Flux.y,
#             mkr1=(marker).y
#            )
# awg.set_wfm(wfm_name=f'{IQList[1]}',
#             wfm=Flux.y
#            )
# awg.upload_wfm
np.linspace(0, 7.5e6, 4)

array([      0., 2500000., 5000000., 7500000.])

In [4]:
# assign waveforms
x_span = 1e-6
freq_start = 0
freq_stop = 7.5e6
num_cases = 4
sampling_rate=1.0E9
freq_range = np.linspace(freq_start, freq_stop, num_cases)
IQList = ['I', 'Q', 'Flux']

const_head = ~Wave(sf.const, [sf.get_x(10e-6, sampling_rate),False])
const_tail = ~Wave(sf.const, [sf.get_x(10e-6, sampling_rate),False])
marker = ~Wave(sf.square, [sf.get_x(x_span, sampling_rate), 0.1e-6, 0.1e-6])

# clear, set, and upload
awg.clr_wfm()
for freq in freq_range:
    c = ~Wave(sf.cosine2, [sf.get_x(x_span, sampling_rate), freq])
    s1 = ~Wave(sf.sine2, [sf.get_x(x_span, sampling_rate), freq])
    s2 = ~(-1 * Wave(sf.sine2, [sf.get_x(x_span, sampling_rate), freq]))
    flux = ~(Wave(sf.const, [sf.get_x(26e-6, sampling_rate), False]))
    
    awg.set_wfm(wfm_name=f'{IQList[0]}_{freq}',
            wfm=(const_head + (2*c)*3 + const_tail).y,
            mkr1=(marker).y
           )
    awg.set_wfm(wfm_name=f'{IQList[1]}_{freq}',
            wfm=(const_head + (s1+s2)*3 + const_tail).y
           )
    awg.set_wfm(wfm_name=f'{IQList[2]}_{freq}',
        wfm=flux.y
       )
awg.upload_wfm()

In [5]:
# assign waveform to corresponding sequence position one at a time
seq_name='freq_mod_iq'
trackList = IQList

# clear, set, assign and upload
awg.clr_seq()
awg.set_seq(seq_name, num_track=len(trackList), num_step=len(freq_range))
#print(awg._seqList['freq_mod_iq'])
for track_idx, track in enumerate(trackList):
    for step_idx, freq in enumerate(freq_range):
        awg.assign_seq(f'{track}_{freq}', seq_name, track_idx+1,step_idx+1)
awg.upload_seq(seq_name)

In [6]:
# assign waveforms/sequences to channels
awg.assign_ch(1, 'freq_mod_iq', track_index=1)
awg.assign_ch(2, 'freq_mod_iq', track_index=2)
awg.assign_ch(3, 'freq_mod_iq', track_index=3)
awg.assign_ch(4, 'freq_mod_iq', track_index=2, auto_output=False, auto_mkr=False)

In [7]:
# set channel amplitudes
awg.set_ch_amp(ch=1, wfm_Vpp=1.0, wfm_offset=0., mkr1=1.0, mkr2=1.0, mkr3=1.0, mkr4=1.0)
awg.set_ch_amp(ch=2, wfm_Vpp=1.0, wfm_offset=0., mkr1=1.0, mkr2=1.0, mkr3=1.0, mkr4=1.0)
awg.set_ch_amp(ch=3, wfm_Vpp=1.5, wfm_offset=0., mkr1=1.0, mkr2=1.0, mkr3=1.0, mkr4=1.0)
awg.set_ch_amp(ch=4, wfm_Vpp=1.5, wfm_offset=0., mkr1=1.0, mkr2=1.0, mkr3=1.0, mkr4=1.0)

In [8]:
print(awg)
#del awg

AWG5208 @ TCPIP0::192.168.10.42::inst0::INSTR
---
Waveform list: dict_keys(['I_0.0', 'Q_0.0', 'Flux_0.0', 'I_2500000.0', 'Q_2500000.0', 'Flux_2500000.0', 'I_5000000.0', 'Q_5000000.0', 'Flux_5000000.0', 'I_7500000.0', 'Q_7500000.0', 'Flux_7500000.0'])
Sequence list: dict_keys(['freq_mod_iq'])
AWG Error state: 0,"No error"
